In [31]:
import pandas as pd
import os

the generalized clean function that includes changing the date when replacing '24:00:00' with '00:00:00'

In [32]:
#want this to work for all files: hms, daily, hourly
def clean_df(df):

    #clean columns/rows
    df.columns = df.iloc[0]
    df = df[3:]
    df.columns = df.columns.str.strip()
    df = df.drop(columns=['Ordinate'])
    df = df.rename(columns={'Date': 'Day', 'Time': 'Time'})
    
    # Increment the date by 1 day where Time is '24:00:00' and replace '24:00:00' with '00:00:00'
    mask = df['Time'] == '24:00:00'
    df.loc[mask, 'Day'] = (pd.to_datetime(df.loc[mask, 'Day'], format='%d-%b-%y') + pd.Timedelta(days=1)).dt.strftime('%d-%b-%y')
    df['Time'] = df['Time'].replace('24:00:00', '00:00:00')

    # Combine 'Day' and 'Time' columns to create a new 'date' column, make sure duplicated columns are dropped
    df['date'] = pd.to_datetime(df['Day'], format='%d-%b-%y') + pd.to_timedelta(df['Time'])
    df.dropna(subset=['date'], inplace=True)
    df = df.loc[:, ~df.columns.duplicated(keep=False)]
    df.set_index('date', inplace=True)

    # Drop 'Day' and 'Time' columns
    df.drop(columns=['Day', 'Time'], inplace=True)
    return df

load the data, double check current directory:

In [33]:
#check current directory
print("Current working directory: ", os.getcwd())
#read in data using calpella as a test case for physics files
hms_hourly = pd.read_csv('data/updated_russian_river_folder/Calpella_hourly.csv')
hms_daily = pd.read_csv('data/updated_russian_river_folder/Calpella_daily.csv')
#read in daily and hourly data
daily = pd.read_csv('data/updated_russian_river_folder/daily.csv')
hourly = pd.read_csv('data/updated_russian_river_folder/hourly.csv')

Current working directory:  /Users/mia/research/UCB-USACE-LSTMs


/var/folders/37/ck6mkvd952v7g_qhcgs0fbrh0000gq/T/ipykernel_26817/1677179345.py:4: DtypeWarning: Columns (0,3,4,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  hms_hourly = pd.read_csv('data/updated_russian_river_folder/Calpella_hourly.csv')
/var/folders/37/ck6mkvd952v7g_qhcgs0fbrh0000gq/T/ipykernel_26817/1677179345.py:8: DtypeWarning: Columns (0,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55) have mixed types. Specify dtype option on import or set low_memory=False.
  hourly = pd.read_csv('data/updated_russian_river_folder/hourly.csv')


view the original df then compare with cleaned df for each df, pay attention to when the date gets changed:

In [34]:
#calpella hourly data not cleaned
hms_hourly.head(30)

,Col number,Unnamed: 1,Unnamed: 2,1,2,3,4,5,6,7
0,Ordinate,Date,Time,Capella Gage FLOW,EF Russian 20 ET-POTENTIAL,EF Russian 20 FLOW,EF Russian 20 FLOW-BASE,EF Russian 20 INFILTRATION,EF Russian 20 PERC-SOIL,EF Russian 20 SATURATION FRACTION
1,Units,NaN,,CFS,IN,CFS,CFS,IN,IN,IN/IN
2,Type,NaN,NaN,INST-VAL,PER-CUM,INST-VAL,INST-VAL,PER-CUM,PER-CUM,INST-VAL
3,2,1-Oct-94,01:00:00,586.723761,0,489.723761,489.723761,0,0,0.7
4,3,1-Oct-94,02:00:00,572.217931,0,475.217931,475.217931,0,0,0.7
5,4,1-Oct-94,03:00:00,558.904771,0,461.904771,461.904771,0,0,0.7
6,5,1-Oct-94,04:00:00,546.701684,0,449.701684,449.701684,0,0,0.7
7,6,1-Oct-94,05:00:00,535.521337,0,438.521337,438.521337,0,0,0.7
8,7,1-Oct-94,06:00:00,525.282679,0,428.282679,428.282679,0,0,0.7
9,8,1-Oct-94,07:00:00,515.910504,0,418.910504,418.910504,0,0,0.7


In [35]:
#cleaned calpella hourly data
cleaned_hms_hourly = clean_df(hms_hourly)
cleaned_hms_hourly.head(30)

,Capella Gage FLOW,EF Russian 20 ET-POTENTIAL,EF Russian 20 FLOW,EF Russian 20 FLOW-BASE,EF Russian 20 INFILTRATION,EF Russian 20 PERC-SOIL,EF Russian 20 SATURATION FRACTION
date,,,,,,,
1994-10-01 01:00:00,586.723761,0,489.723761,489.723761,0,0,0.7
1994-10-01 02:00:00,572.217931,0,475.217931,475.217931,0,0,0.7
1994-10-01 03:00:00,558.904771,0,461.904771,461.904771,0,0,0.7
1994-10-01 04:00:00,546.701684,0,449.701684,449.701684,0,0,0.7
1994-10-01 05:00:00,535.521337,0,438.521337,438.521337,0,0,0.7
1994-10-01 06:00:00,525.282679,0,428.282679,428.282679,0,0,0.7
1994-10-01 07:00:00,515.910504,0,418.910504,418.910504,0,0,0.7
1994-10-01 08:00:00,507.335048,0.001547,410.335048,410.335048,0,0,0.699845
1994-10-01 09:00:00,499.491614,0.004158,402.491614,402.491614,0,0,0.699429


In [36]:
#calpella daily data not cleaned
hms_daily.head(30)

,Col Number,Unnamed: 1,Unnamed: 2,1,2,3,4,5,6,7
0,Ordinate,Date,Time,Capella Gage FLOW,EF Russian 20 ET-POTENTIAL,EF Russian 20 FLOW,EF Russian 20 FLOW-BASE,EF Russian 20 INFILTRATION,EF Russian 20 PERC-SOIL,EF Russian 20 SATURATION FRACTION
1,Units,NaN,NaN,CFS,IN,CFS,CFS,IN,IN,IN/IN
2,Type,NaN,NaN,PER-AVER,PER-CUM,PER-AVER,PER-AVER,PER-CUM,PER-CUM,PER-AVER
3,1,1-Oct-94,24:00:00,481.916626,0.066151,385.320801,385.320801,0,0,0.696838
4,2,2-Oct-94,24:00:00,422.200256,0.110752,325.200256,325.200256,0,0,0.687917
5,3,3-Oct-94,24:00:00,401.010406,0.097419,301.072906,301.072906,0,0,0.677802
6,4,4-Oct-94,24:00:00,350.536926,0.026083,271.099426,271.099426,0,0,0.671157
7,5,5-Oct-94,24:00:00,325.131805,0.053415,237.319305,237.319305,0,0,0.670204
8,6,6-Oct-94,24:00:00,292.534271,0.086248,204.534256,204.534256,0,0,0.667322
9,7,7-Oct-94,24:00:00,262.955078,0.091789,174.955078,174.955078,0,0,0.659376


In [37]:
#cleaned calpella hourly data
cleaned_hms_daily = clean_df(hms_daily)
cleaned_hms_daily.head(30)

,Capella Gage FLOW,EF Russian 20 ET-POTENTIAL,EF Russian 20 FLOW,EF Russian 20 FLOW-BASE,EF Russian 20 INFILTRATION,EF Russian 20 PERC-SOIL,EF Russian 20 SATURATION FRACTION
date,,,,,,,
1994-10-02,481.916626,0.066151,385.320801,385.320801,0,0,0.696838
1994-10-03,422.200256,0.110752,325.200256,325.200256,0,0,0.687917
1994-10-04,401.010406,0.097419,301.072906,301.072906,0,0,0.677802
1994-10-05,350.536926,0.026083,271.099426,271.099426,0,0,0.671157
1994-10-06,325.131805,0.053415,237.319305,237.319305,0,0,0.670204
1994-10-07,292.534271,0.086248,204.534256,204.534256,0,0,0.667322
1994-10-08,262.955078,0.091789,174.955078,174.955078,0,0,0.659376
1994-10-09,237.137482,0.085098,149.137482,149.137482,0,0,0.65047
1994-10-10,212.975372,0.081256,126.933701,126.933701,0,0,0.642361


In [38]:
#check daily data
daily.head(30)

,col num,Unnamed: 1,Unnamed: 2,1,2,3,4,5,6,7,...,44,45,46,47,48,49,50,51,52,53
0,Ordinate,Date,Time,AUSTIN CR ET-POTENTIAL RUN:BASIN AVERAGE 60 YR,BIG SULPHUR CR ET-POTENTIAL RUN:BASIN AVERAGE ...,DRY CREEK 10 ET-POTENTIAL RUN:BASIN AVERAGE 60 YR,DRY CREEK 20 ET-POTENTIAL RUN:BASIN AVERAGE 60 YR,DRY CREEK 30 ET-POTENTIAL RUN:BASIN AVERAGE 60 YR,EF RUSSIAN 10 ET-POTENTIAL RUN:BASIN AVERAGE 6...,EF RUSSIAN 20 ET-POTENTIAL RUN:BASIN AVERAGE 6...,...,UKIAH CA SOLAR RADIATION USAF-NOAA,UKIAH CA TEMPERATURE USAF-NOAA,UKIAH CA WINDSPEED USAF-NOAA,POTTER VALLEY CA FLOW USGS_ADJUSTED,UKIAH CA FLOW USGS-MERGED,GEYSERVILLE CA FLOW USGS-MERGED,NR CALPELLA FLOW COE CPL,NR HOPLAND FLOW COE HOP,LAKE SONOMA FLOW-RES IN CALC-VAL-SHIFT-SMOOTH,NR GUERNEVILLE FLOW COE GRN
1,Units,,,IN,IN,IN,IN,IN,IN,IN,...,WATT/M2,DEG F,MPH,CFS,CFS,CFS,CFS,CFS,CFS,CFS
2,Type,,,PER-CUM,PER-CUM,PER-CUM,PER-CUM,PER-CUM,PER-CUM,PER-CUM,...,PER-AVER,PER-AVER,PER-AVER,PER-AVER,PER-AVER,PER-AVER,PER-AVER,PER-AVER,PER-AVER,PER-AVER
3,1,1-Oct-94,24:00:00,0.111,0.0949,0.1109,0.111,0.0946,0.0945,0.0945,...,134.6417,67.0175,1.125,97,153,84,NaN,NaN,NaN,NaN
4,2,2-Oct-94,24:00:00,0.1567,0.1585,0.1567,0.1567,0.1583,0.1582,0.1582,...,213.225,68.8175,2.125,97,153,84,NaN,NaN,NaN,NaN
5,3,3-Oct-94,24:00:00,0.1341,0.1395,0.134,0.134,0.1393,0.1391,0.1392,...,214.4,62.1275,5.0625,100,153,93,NaN,NaN,NaN,NaN
6,4,4-Oct-94,24:00:00,0.0374,0.0374,0.0373,0.0374,0.0373,0.0372,0.0373,...,73.5458,52.76,2.625,79,139,101,NaN,NaN,NaN,NaN
7,5,5-Oct-94,24:00:00,0.1107,0.0765,0.1107,0.1107,0.0764,0.0763,0.0763,...,129.1375,55.4075,2.3958,88,123,102,NaN,NaN,NaN,NaN
8,6,6-Oct-94,24:00:00,0.1216,0.1235,0.1215,0.1215,0.1233,0.1232,0.1232,...,219.8583,57.9237,2,88,119,102,NaN,NaN,NaN,NaN
9,7,7-Oct-94,24:00:00,0.1425,0.1314,0.1425,0.1425,0.1312,0.1311,0.1311,...,217.7667,61.3063,1.25,88,119,101,NaN,NaN,NaN,NaN


In [39]:
clean_daily = clean_df(daily)
clean_daily.head(30)

,AUSTIN CR ET-POTENTIAL RUN:BASIN AVERAGE 60 YR,BIG SULPHUR CR ET-POTENTIAL RUN:BASIN AVERAGE 60 YR,DRY CREEK 10 ET-POTENTIAL RUN:BASIN AVERAGE 60 YR,DRY CREEK 20 ET-POTENTIAL RUN:BASIN AVERAGE 60 YR,DRY CREEK 30 ET-POTENTIAL RUN:BASIN AVERAGE 60 YR,EF RUSSIAN 10 ET-POTENTIAL RUN:BASIN AVERAGE 60 YR,EF RUSSIAN 20 ET-POTENTIAL RUN:BASIN AVERAGE 60 YR,GREEN VALLEY ET-POTENTIAL RUN:BASIN AVERAGE 60 YR,LAGUNA ET-POTENTIAL RUN:BASIN AVERAGE 60 YR,RUSSIAN 10 ET-POTENTIAL RUN:BASIN AVERAGE 60 YR,...,UKIAH CA SOLAR RADIATION USAF-NOAA,UKIAH CA TEMPERATURE USAF-NOAA,UKIAH CA WINDSPEED USAF-NOAA,POTTER VALLEY CA FLOW USGS_ADJUSTED,UKIAH CA FLOW USGS-MERGED,GEYSERVILLE CA FLOW USGS-MERGED,NR CALPELLA FLOW COE CPL,NR HOPLAND FLOW COE HOP,LAKE SONOMA FLOW-RES IN CALC-VAL-SHIFT-SMOOTH,NR GUERNEVILLE FLOW COE GRN
date,,,,,,,,,,,,,,,,,,,,,
1994-10-02,0.111,0.0949,0.1109,0.111,0.0946,0.0945,0.0945,0.1109,0.111,0.1109,...,134.6417,67.0175,1.125,97,153,84,NaN,NaN,NaN,NaN
1994-10-03,0.1567,0.1585,0.1567,0.1567,0.1583,0.1582,0.1582,0.1567,0.1568,0.1567,...,213.225,68.8175,2.125,97,153,84,NaN,NaN,NaN,NaN
1994-10-04,0.1341,0.1395,0.134,0.134,0.1393,0.1391,0.1392,0.1341,0.1341,0.1341,...,214.4,62.1275,5.0625,100,153,93,NaN,NaN,NaN,NaN
1994-10-05,0.0374,0.0374,0.0373,0.0374,0.0373,0.0372,0.0373,0.0374,0.0374,0.0374,...,73.5458,52.76,2.625,79,139,101,NaN,NaN,NaN,NaN
1994-10-06,0.1107,0.0765,0.1107,0.1107,0.0764,0.0763,0.0763,0.1107,0.1108,0.1107,...,129.1375,55.4075,2.3958,88,123,102,NaN,NaN,NaN,NaN
1994-10-07,0.1216,0.1235,0.1215,0.1215,0.1233,0.1232,0.1232,0.1215,0.1216,0.1215,...,219.8583,57.9237,2,88,119,102,NaN,NaN,NaN,NaN
1994-10-08,0.1425,0.1314,0.1425,0.1425,0.1312,0.1311,0.1311,0.1425,0.1426,0.1425,...,217.7667,61.3063,1.25,88,119,101,NaN,NaN,NaN,NaN
1994-10-09,0.1235,0.1219,0.1234,0.1234,0.1217,0.1215,0.1216,0.1235,0.1236,0.1235,...,189.0708,62.8925,1.25,88,119,100,NaN,NaN,NaN,NaN
1994-10-10,0.0884,0.1164,0.0883,0.0884,0.1162,0.116,0.1161,0.0884,0.0884,0.0884,...,144.5208,62.9,2.5833,86,119,100,NaN,NaN,NaN,NaN


In [40]:
#check hourly data
hourly.head(30)

,col num,Unnamed: 1,Unnamed: 2,1,2,3,4,5,6,7,...,44,45,46,47,48,49,50,51,52,53
0,Ordinate,Date,Time,AUSTIN CR ET-POTENTIAL RUN:BASIN AVERAGE 60 YR,BIG SULPHUR CR ET-POTENTIAL RUN:BASIN AVERAGE ...,DRY CREEK 10 ET-POTENTIAL RUN:BASIN AVERAGE 60 YR,DRY CREEK 20 ET-POTENTIAL RUN:BASIN AVERAGE 60 YR,DRY CREEK 30 ET-POTENTIAL RUN:BASIN AVERAGE 60 YR,EF RUSSIAN 10 ET-POTENTIAL RUN:BASIN AVERAGE 6...,EF RUSSIAN 20 ET-POTENTIAL RUN:BASIN AVERAGE 6...,...,UKIAH CA SOLAR RADIATION USAF-NOAA,UKIAH CA TEMPERATURE USAF-NOAA,UKIAH CA WINDSPEED USAF-NOAA,POTTER VALLEY CA FLOW USGS_ADJUSTED,UKIAH CA FLOW USGS-MERGED,GEYSERVILLE CA FLOW USGS-MERGED,NR CALPELLA FLOW COE CPL,NR HOPLAND FLOW COE HOP,LAKE SONOMA FLOW-RES IN CALC-VAL-SHIFT-SMOOTH,NR GUERNEVILLE FLOW COE GRN
1,Units,NaN,NaN,IN,IN,IN,IN,IN,IN,IN,...,WATT/M2,DEG F,MPH,CFS,CFS,CFS,CFS,CFS,CFS,CFS
2,Type,,NaN,PER-CUM,PER-CUM,PER-CUM,PER-CUM,PER-CUM,PER-CUM,PER-CUM,...,PER-AVER,INST-VAL,INST-VAL,PER-AVER,PER-AVER,PER-AVER,PER-AVER,PER-AVER,PER-AVER,PER-AVER
3,2,1-Oct-94,01:00:00,0,0,0,0,0,0,0,...,0,57.92,0,97,153,84,NaN,NaN,NaN,NaN
4,3,1-Oct-94,02:00:00,0,0,0,0,0,0,0,...,0,55.94,0,97,153,84,NaN,NaN,NaN,NaN
5,4,1-Oct-94,03:00:00,0,0,0,0,0,0,0,...,0,54.86,0,97,153,84,NaN,NaN,NaN,NaN
6,5,1-Oct-94,04:00:00,0,0,0,0,0,0,0,...,0,53.96,0,97,153,84,NaN,NaN,NaN,NaN
7,6,1-Oct-94,05:00:00,0,0,0,0,0,0,0,...,0,52.88,0,97,153,84,NaN,NaN,NaN,NaN
8,7,1-Oct-94,06:00:00,0,0,0,0,0,0,0,...,0,52.88,0,97,153,84,NaN,NaN,NaN,NaN
9,8,1-Oct-94,07:00:00,0,0,0,0,0,0,0,...,53.8,51.98,0,97,153,84,NaN,NaN,NaN,NaN


In [41]:
cleaned_hourly = clean_df(hourly)
cleaned_hourly.head(30)

,AUSTIN CR ET-POTENTIAL RUN:BASIN AVERAGE 60 YR,BIG SULPHUR CR ET-POTENTIAL RUN:BASIN AVERAGE 60 YR,DRY CREEK 10 ET-POTENTIAL RUN:BASIN AVERAGE 60 YR,DRY CREEK 20 ET-POTENTIAL RUN:BASIN AVERAGE 60 YR,DRY CREEK 30 ET-POTENTIAL RUN:BASIN AVERAGE 60 YR,EF RUSSIAN 10 ET-POTENTIAL RUN:BASIN AVERAGE 60 YR,EF RUSSIAN 20 ET-POTENTIAL RUN:BASIN AVERAGE 60 YR,GREEN VALLEY ET-POTENTIAL RUN:BASIN AVERAGE 60 YR,LAGUNA ET-POTENTIAL RUN:BASIN AVERAGE 60 YR,RUSSIAN 10 ET-POTENTIAL RUN:BASIN AVERAGE 60 YR,...,UKIAH CA SOLAR RADIATION USAF-NOAA,UKIAH CA TEMPERATURE USAF-NOAA,UKIAH CA WINDSPEED USAF-NOAA,POTTER VALLEY CA FLOW USGS_ADJUSTED,UKIAH CA FLOW USGS-MERGED,GEYSERVILLE CA FLOW USGS-MERGED,NR CALPELLA FLOW COE CPL,NR HOPLAND FLOW COE HOP,LAKE SONOMA FLOW-RES IN CALC-VAL-SHIFT-SMOOTH,NR GUERNEVILLE FLOW COE GRN
date,,,,,,,,,,,,,,,,,,,,,
1994-10-01 01:00:00,0,0,0,0,0,0,0,0,0,0,...,0,57.92,0,97,153,84,NaN,NaN,NaN,NaN
1994-10-01 02:00:00,0,0,0,0,0,0,0,0,0,0,...,0,55.94,0,97,153,84,NaN,NaN,NaN,NaN
1994-10-01 03:00:00,0,0,0,0,0,0,0,0,0,0,...,0,54.86,0,97,153,84,NaN,NaN,NaN,NaN
1994-10-01 04:00:00,0,0,0,0,0,0,0,0,0,0,...,0,53.96,0,97,153,84,NaN,NaN,NaN,NaN
1994-10-01 05:00:00,0,0,0,0,0,0,0,0,0,0,...,0,52.88,0,97,153,84,NaN,NaN,NaN,NaN
1994-10-01 06:00:00,0,0,0,0,0,0,0,0,0,0,...,0,52.88,0,97,153,84,NaN,NaN,NaN,NaN
1994-10-01 07:00:00,0,0,0,0,0,0,0,0,0,0,...,53.8,51.98,0,97,153,84,NaN,NaN,NaN,NaN
1994-10-01 08:00:00,0,0,0,0,0,0,0,0,0,0,...,180.2,55.94,0,97,153,84,NaN,NaN,NaN,NaN
1994-10-01 09:00:00,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,...,303.2,61.88,0,97,153,84,NaN,NaN,NaN,NaN
